In [2]:
import librosa
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

# 데이터 로드
df = pd.read_csv("aihub_emotion_labeled_mapped.csv", encoding="cp949", sep="\t")

# 존재하는 파일만 필터링
df['file_exist'] = df['audio_path'].apply(lambda x: os.path.exists(x) if pd.notnull(x) else False)
df = df[df['file_exist'] & df['final_emotion'].notnull()].copy()

# 전처리 기준
TARGET_SR = 16000
RMS_THRESHOLD_DB = -70  # 너무 조용한 경우 제거

def is_valid_audio(path):
    try:
        audio, sr = librosa.load(path, sr=TARGET_SR, mono=True)
        rms_db = librosa.power_to_db(librosa.feature.rms(y=audio), ref=np.max)
        return rms_db.mean() > RMS_THRESHOLD_DB
    except Exception as e:
        print(f"❌ Error loading {path}: {e}")
        return False

# 유효한 오디오만 필터링
tqdm.pandas()
df['valid_signal'] = df['audio_path'].progress_apply(is_valid_audio)

# 최종 전처리 완료 데이터셋
df_clean = df[df['valid_signal']].copy()
print("🎯 전처리 완료! 남은 유효 샘플 수:", len(df_clean))

# 저장
df_clean.to_csv("aihub_emotion_preprocessed.csv", index=False, encoding="utf-8-sig")

100%|██████████| 24938/24938 [00:43<00:00, 573.21it/s]

🎯 전처리 완료! 남은 유효 샘플 수: 24938
